In [1]:
pip install pyamaze

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def DFS(m):
    # The starting point of the maze, in this case, the bottom right corner.
    start=(m.rows,m.cols)

    # List to keep track of the cells we have already explored.
    explored=[start]

    # Stack for implementing the DFS algorithm (Last In, First Out principle).
    frontier=[start]

    # Dictionary to keep track of the path from each node to its parent.
    dfsPath={}

    # List to keep track of the order in which nodes are visited.
    dSearch=[]

    # DFS loop that continues until all reachable nodes have been explored.
    while len(frontier)>0:
        # Pop the last node from the stack (depth-first principle).
        current=frontier.pop()
        dSearch.append(current)

        # If we reached the top left corner (goal), we break the loop.
        if current==m._goal:
            break

        # Check all 4 possible directions from the current cell.
        for d in 'ESNW':
            # If there's no wall in the current direction...
            if m.maze_map[current][d]==True:
                # ...calculate the coordinates of the child cell.
                if d=='E':
                    childCell=(current[0],current[1]+1)
                elif d=='W':
                    childCell=(current[0],current[1]-1)
                elif d=='S':
                    childCell=(current[0]+1,current[1])
                elif d=='N':
                    childCell=(current[0]-1,current[1])

                # If the child cell has already been explored, we ignore it.
                if childCell in explored:
                    continue

                # Otherwise, mark it as explored and add it to the frontier and path.
                explored.append(childCell)
                frontier.append(childCell)
                dfsPath[childCell]=current

    # Construct the shortest path from the start to the goal.
    fwdPath={}
    cell=m._goal
    while cell!=start:
        fwdPath[dfsPath[cell]]=cell
        cell=dfsPath[cell]

    # Return the search, path, and forward path.
    return dSearch,dfsPath,fwdPath

In [3]:
# Import necessary modules.
from collections import deque

def BFS(m, start=None):
    # Define the start cell, if not provided use the bottom-right cell.
    if start is None:
        start = (m.rows, m.cols)

    # Initialize the frontier as a deque (double-ended queue), start with the start cell.
    frontier = deque()
    frontier.append(start)

    # Dictionary to store the path taken by BFS.
    bfsPath = {}

    # List to keep track of the cells that have already been explored.
    explored = [start]

    # List to store the order of cells explored.
    bfsSearch = []

    # Process while there are cells in the frontier.
    while len(frontier) > 0:
        # Dequeue a cell from the frontier.
        currCell = frontier.popleft()

        # If the current cell is the goal, then stop processing.
        if currCell == m._goal:
            break

        # Check each direction from the current cell.
        for d in 'ESNW':
            # If the current direction is passable (no wall)...
            if m.maze_map[currCell][d] == True:
                # Determine the coordinates of the child cell based on the direction.
                if d == 'E':
                    childCell = (currCell[0], currCell[1] + 1)
                elif d == 'W':
                    childCell = (currCell[0], currCell[1] - 1)
                elif d == 'S':
                    childCell = (currCell[0] + 1, currCell[1])
                elif d == 'N':
                    childCell = (currCell[0] - 1, currCell[1])

                # If the child cell has already been explored, skip this iteration.
                if childCell in explored:
                    continue

                # Add the child cell to the frontier and mark it as explored.
                frontier.append(childCell)
                explored.append(childCell)

                # Add the child cell and its parent to the BFS path. Also, update the search order list.
                bfsPath[childCell] = currCell
                bfsSearch.append(childCell)

    # Trace the path from the goal to the start.
    fwdPath = {}
    cell = m._goal
    while cell != (m.rows, m.cols):
        fwdPath[bfsPath[cell]] = cell
        cell = bfsPath[cell]

    # Return the search order, path, and forward path.
    return bfsSearch, bfsPath, fwdPath

In [4]:
# Import necessary modules and functions
from pyamaze import maze, agent, COLOR, textLabel  # Pyamaze library for creating and visualizing mazes
from timeit import timeit  # Function for measuring execution time

# Create a maze instance
m = maze()

# Load the maze layout from a CSV file
m.CreateMaze(loadMaze='mazemain.csv')

# Run the DFS algorithm on the maze and get the paths
searchPath, dfsPath, fwdDFSPath = DFS(m)

# Run the BFS algorithm on the maze and get the paths
bSearch, bfsPath, fwdBFSPath = BFS(m)

# Display the lengths of the paths found by DFS and BFS on the maze
textLabel(m, 'DFS Path Length', len(fwdDFSPath)+1)
textLabel(m, 'BFS Path Length', len(fwdBFSPath)+1)
textLabel(m, 'DFS Search Length', len(searchPath)+1)
textLabel(m, 'BFS Search Length', len(bSearch)+1)

# Create two agents for visualizing the paths
a = agent(m, footprints=True, color=COLOR.cyan, filled=True)  # Agent for BFS
b = agent(m, footprints=True, color=COLOR.yellow)  # Agent for DFS

# Have the agents trace the paths found by BFS and DFS
m.tracePath({b: fwdDFSPath}, delay=80)
m.tracePath({a: fwdBFSPath}, delay=80)


# Measure the execution time of DFS and BFS
t1 = timeit('DFS(m)', number=100, globals=globals())  # Time for DFS
t2 = timeit('BFS(m)', number=100, globals=globals())  # Time for BFS

# Display the execution times on the maze
textLabel(m, 'DFS Time', t1)
textLabel(m, 'BFS Time', t2)

# Start the visualization
m.run()